### LSTM

В данном ноутбуке приведен код для экспериментов с LSTM сетью. 

Для проверки экспериментов необходимо скачать предобученную модель с сайта

https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
def Concat_data(D, age):
    label = np.zeros(D.shape[0]*(D.shape[0]-1))
    
    Data_a = np.zeros((D.shape[0]*(D.shape[0]-1), 140, 300))
    Data_b = np.zeros((D.shape[0]*(D.shape[0]-1), 140, 300))
    L = []
    
    ind = 0
    for i in range(D.shape[0]):
        for j in range(D.shape[0]):
            if i != j:
                
            #Data.iloc[ind] = D['text'][i], D['title'][i], D['text'][j], D['title'][j], D['age'][i] >  D['age'][j]
                Data_a[ind] = D[i]
                Data_b[ind] = D[j]
                label[ind] = age[i]>age[j]
                ind += 1
                
    return Data_a, Data_b, label

In [ ]:
Data = pd.read_csv('lyric.csv', header=None)
name_and_year = Data[0].str.split('-')
name = []
year = []
title = []
for s in name_and_year:
    name.append(s[0])
    year.append(s[1][-5:-1])
    title.append(s[1][:-7])
    
Data['title'] = title
Data['author_name'] = name
Data['year'] = Data[0].str[-5:-1].astype('int')
Data.columns = ['inf', 'text', 'title', 'author_name', 'year']
Data.drop(np.array(range(Data.shape[0]))[np.array(Data.text.str.count(' ')<5)], axis = 0, inplace=True)
Data.index = np.array(range(Data.shape[0]))
D = Data.loc[Data.text.str.len() < 1014, :][Data.columns[1:]]
D.index = np.array(range(D.shape[0]))
D['name'] = pd.DataFrame(list(D.author_name.str.split(' ')), index = None)[0]
D['surname'] = pd.DataFrame(list(D.author_name.str.split(' ')), index = None)[1]
bithday = pd.read_csv('years.txt', sep = ' ', header = None)
bithday.columns = ['surname', 'name', 'year_bith']
D = D.merge(bithday, on=["surname", "name"])
D.index = np.array(range(D.shape[0]))

D.year = D.year.astype('int')
D['age'] = D.year - D.year_bith

D.text = D.text.str.lower()
D.text = D.text.str.replace('\n', ' ')

import string

translator = str.maketrans('', '', string.punctuation)
D.text = D.text.str.translate(translator)

D.text = D.text.str.replace('  ', ' ')
D.text = D.text.str.replace('  ', ' ')
D.text = D.text.str.replace('  ', ' ')
D.text = D.text.str.replace('  ', ' ')
D.text = D.text.str.replace('  ', ' ')

D.text = D.text.str.split(' ')

In [ ]:
import gensim

model = gensim.models.Word2Vec.load('model_w2v',mmap='r')

In [ ]:
data = np.zeros((D.text.shape[0], 140,  300))
i = 0
for text in D.text:
    j = 0
    for word in text:
        if j > 0:
            try:
                data[i, j] = model.wv[word]
            except:
                j -= 1
        j += 1
    i += 1

In [ ]:
r = 0# for r in range(D.author_name.unique().shape[0])

from sklearn.model_selection import LeaveOneGroupOut
from sklearn import ensemble
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import linear_model
clf = sklearn.preprocessing.LabelEncoder()
clf.fit(D.author_name)
label = clf.transform(D.author_name)


from sklearn.model_selection import train_test_split


groups = np.zeros(label.shape).astype('int')
X = D.copy()
Y = X['age']
#X.drop('age', axis = 1, inplace = True)

#clf.fit(X[:70000], Y[:70000])
X_train, X_test, y_train, y_test = train_test_split(data[label == r], Y[label == r], test_size=0.2, random_state=2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
data_a, data_b, label = Concat_data(X_train, np.array(y_train))
data_test_a, data_test_b, lab = Concat_data(X_test, np.array(y_test))

### LSTM 

In [ ]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

tweet_a = Input(shape=(140, 300))
tweet_b = Input(shape=(140, 300))
# This layer can take as input a matrix
# and will return a vector of size 64
shared_lstm = LSTM(64)

# When we reuse the same layer instance
# multiple times, the weights of the layer
# are also being reused
# (it is effectively *the same* layer)
encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

# We can then concatenate the two vectors:
merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)

# And add a logistic regression on top
predictions = Dense(1, activation='sigmoid')(merged_vector)

# We define a trainable model linking the
# tweet inputs to the predictions
model = Model(inputs=[tweet_a, tweet_b], outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit([data_a, data_b], label, epochs=10)